In [1]:
import numpy as np
from scipy.special import expit
from evaluator_Linear import evaluator
from probLearner import PMLearner, RewardLearner, PALearner
from ratioLearner import  RatioLinearLearner as RatioLearner
from qLearner_Linear import Qlearner

1. Data structure update: add a new key "time_idx"; need to specify the num_trajectory and num_time (make sure that the data is structured as $[Obs_{00},Obs_{01},\cdots,Obs_{NT}]$)
2. three target policies are of interest
    - (**Important!**) optimal policy (need to learn the behavior policy first, and save as a pickle file) (t_depend_target = False)
    - (**Important!**) behavior policy (t_depend_target = False)
    - contrast policies at different time t (t_depend_target = True)
3. two versions of state are of interest
    - $S_t = [mood_{t-1}]$
    - (not consider for now) $S_t = [mood_{t-1}, step_{t-1}, sleep_{t-1}]$
4. two versions of Q models are of interest
    - Q = f([1,s]) (t_dependent_Q = False)
    - Q = f([1,s,t] (t_dependent_Q = True)
5. other parameters can be modified to improve the performance
    - ratio_ndim = 15 (for S=mood, lower the ratio_ndim)
    - d = 3
    - L = 7
    - scaler: 'Identity',"NormCdf","Standardize"

## STEP0： Define the Target Policy and the Control Policy

In [2]:
# Control Policy
def control_policy(state = None, dim_state=None, action=None, get_a = False):
    # fixed policy with fixed action 0
    if get_a:
        action_value = np.array([0])
    else:
        state = np.copy(state).reshape(-1,dim_state)
        NT = state.shape[0]
        if action is None:
            action_value = np.array([0]*NT)
        else:
            action = np.copy(action).flatten()
            if len(action) == 1 and NT>1:
                action = action * np.ones(NT)
            action_value = 1-action
    return action_value


In [ ]:
# target_policy 1: Optimal Policy Learned from Observational Data
from qbehavior import Learn_Behavior_Q
from _util import *
import pickle
#"Q_behavior.pickle" is the file name we saved the behavior Q estimations
Q_behavior = pickle.load(open("Q_behavior.pickle", "rb", -1))
def optimal_policy(state = None, dim_state = 1, action=None): 
    opt_A = Q_behavior.opt_A(state) 
    if action is None:
        action_value = opt_A
    else:
        action = np.copy(action).flatten()
        action_value = 1-abs(opt_A-action) 
    return action_value

In [3]:
# target_policy 2: Series of t-dependent policies, i.e., policy_0 = [1,0,0,0,.....]
# set trt_step as 0/1/2/3/..../24
trt_step = 0
def contrast_policy_1(state = None, dim_state = 1, action=None, time_idx = None):
    time_idx = np.copy(time_idx).reshape((-1,1))
    NT = time_idx.shape[0]
    A = 0+(time_idx == trt_step)
    A = A.flatten()
    if action is None:
        action_value = A
    else:
        action = np.copy(action).flatten()
        action_value = 1-abs(A-action) 
    return action_value

In [4]:
# target_policy 3: behavior policy
def behavior_policy(state, dim_state = 1, action=None):
    state = np.copy(state).reshape((-1, dim_state))
    NT = state.shape[0]
    pa = .75 * np.ones(NT)
    if action is None:
        if NT == 1:
            pa = pa[0]
            prob_arr = np.array([1-pa, pa])
            action_value = np.random.choice([0, 1], 1, p=prob_arr)
        else:
            raise ValueError('No random for matrix input')
    else:
        action = np.copy(action).flatten()
        action_value = pa * action + (1-pa) * (1-action)
    return action_value

## STEP1: prepare the dataset

The following is an example of a proper input dataset with 2 trajectories and 3 observations for each trajectory, which is a dictionary with keys:
- s0: stacked initial states of all the trajectories, initial state, 2d-array
- state: stacked states of all the trajectories at all time points, 2d-array
- action: stacked sequence of actions for all trajectories at all time points, 1d-array
- mediator: stacked mediators of all the trajectories at all time points, 2d-array
- reward: stacked sequence of rewards for all trajectories at all time points, 1d-array
- next_state: stacked next_states of all the trajectories at all time points, 2d-array
- **time_idx: stacked time step index, 1d-array**

In [6]:
#from Simulator import Simulator
#dim_state=1; dim_mediator = 2
#std_M = 2; std_S = 2
#simulator = Simulator(model_type='Gaussian_semi', dim_state=dim_state, 
#                      dim_mediator = dim_mediator, std_M = std_M, std_S = std_S)
#simulator.sample_trajectory(num_trajectory=30, num_time=30, seed=0)
#simulator.trajectory2iid()
#sim_iid_dataset = simulator.iid_dataset
#dataset = sim_iid_dataset
#dataset

## STEP2: Modify the hyper-parameters

In [8]:
#Fixed hyper-parameter--no need to modify
expectation_MCMC_iter = 50
expectation_MCMC_iter_Q3 = expectation_MCMC_iter_Q_diff = 50
truncate = 50
problearner_parameters = {"splitter":["best","random"], "max_depth" : range(1,50)},

#hyperparameters that need modification
#dim_state = the dimension of the state variable
#dim_meditor = the dimension of the mediator variable
#ratio_ndim = number of features used to learn the ratio model # can be modified accordingly 
                #to learn how the ratio_ndim affect the estimation performance
dim_state=1; dim_mediator = 2

ratio_ndim = 15
d = 3
L = 7

t_depend_target = False
target_policy= behavior_policy
control_policy = control_policy
t_dependent_Q = False
scaler = 'Identity'
num_trajectory = 30
num_time = 30

## STEP3: Causal Effect Estimation (target policy = behavior policy)

In [9]:
est_obj1 = evaluator(dataset, num_trajectory, num_time,
                     Qlearner, PMLearner, RewardLearner, PALearner, RatioLearner,
                     problearner_parameters = problearner_parameters,
                     ratio_ndim = ratio_ndim, truncate = truncate, l2penalty = 10**(-4),
                     t_depend_target = t_depend_target,
                     target_policy=target_policy, control_policy = control_policy, 
                     dim_state = dim_state, dim_mediator = dim_mediator, 
                     Q_settings = {'scaler': scaler,'product_tensor': False, 'beta': 3/7, 
                                   'include_intercept': False, 'expectation_MCMC_iter_Q3': expectation_MCMC_iter_Q3, 
                                   'expectation_MCMC_iter_Q_diff':expectation_MCMC_iter_Q_diff, 
                                   'penalty': 10**(-4),'d': d, 'min_L': L, "t_dependent_Q": t_dependent_Q},
                     expectation_MCMC_iter = expectation_MCMC_iter,
                     seed = 10)

est_obj1.estimate_DE_ME_SE()
est_value1 = est_obj1.est_DEMESE
se_value1 = est_obj1.se_DEMESE


#The following are the estimations of our interest

#1. estimation used the proposed triply robust estimator
IDE_MR, IME_MR, DDE_MR, DME_MR = est_value1[:4]

#2. estimation used the direct estimator of etas
IDE_Direct, IME_Direct, DDE_Direct, DME_Direct = est_value1[4:8]

#3. estimation used the WIS1 estimator of etas
IDE_WIS1, IME_WIS1, DDE_WIS1, DME_WIS1 = est_value1[8:12]

#4. estimation used the WIS2 estimator of etas
IDE_WIS2, IME_WIS2 = est_value1[12:14]

#5. estimation used the baseline method
IDE_baseline, IME_baseline = est_value1[14:16]

#6. SE of each estimator
se_value1

Building 0-th basis spline (total 3 state-mediator dimemsion) which has 3 basis, in total 3 features 
Building 1-th basis spline (total 3 state-mediator dimemsion) which has 3 basis, in total 6 features 
Building 2-th basis spline (total 3 state-mediator dimemsion) which has 3 basis, in total 9 features 


array([1.50883687e-01, 1.81103125e-01, 1.30194736e-01, 1.38430969e-01,
       2.43237678e-16, 1.62158452e-16, 4.05396130e-17, 0.00000000e+00,
       4.31876076e-01, 2.75846953e-01, 6.92016723e-02, 1.01059660e-01,
       1.62981134e-01, 2.81397155e-01, 4.44346548e-02, 2.78994450e-02])